In [1]:
#Install the Required Libraries:
!pip install Flask scikit-learn requests beautifulsoup4 nltk transformers pdfminer.six python-docx


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for pdfminer.six from https://files.pythonhosted.org/packages/67/7d/44d6b90e5a293d3a975cefdc4e12a932ebba814995b2a07e37e599dd27c6/pdfminer.six-20240706-py3-none-any.whl.metadata
  Obtaining dependency information for python-docx from https://files.pythonhosted.org/packages/3e/3d/330d9efbdb816d3f60bf2ad92f05e1708e4a1b9abe80461ac3444c83f749/python_docx-1.1.2-py3-none-any.whl.metadata
  Obtaining dependency information for typing-extensions>=4.9.0 from https://files.pythonhosted.org/packages/26/9f/ad63fc0248c5379346306f8668cda6e2e2e9c95e01216d2b8ffd9ff037d0/typing_extensions-4.12.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.6 MB 131.3 kB/s eta

In [2]:
#Import the Libraries:
import re
import requests
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\thila\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [3]:
#Extract from PDF
from pdfminer.high_level import extract_text

def extract_text_from_pdf(pdf_path):
    text = extract_text(pdf_path)
    return text


In [4]:
#Extract from docx
from docx import Document

def extract_text_from_docx(docx_path):
    doc = Document(docx_path)
    text = "\n".join([para.text for para in doc.paragraphs])
    return text


In [21]:
#Clean Text
import re
from nltk.tokenize import word_tokenize

def clean_text(text):
    # Check if the input is a list and join it into a single string
    if isinstance(text, list):
        text = ' '.join(text)
    
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    
    # Convert to lowercase and tokenize
    tokens = word_tokenize(text.lower())
    
    return tokens


In [6]:
#Convert Text to TF-IDF Vectors:
#TF-IDF (Term Frequency-Inverse Document Frequency) is a way to represent text in numerical form:
def vectorize_text(text_list):
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(text_list)
    return vectors


In [7]:
def calculate_cosine_similarity(vectors):
    cosine_matrix = cosine_similarity(vectors)
    return cosine_matrix


In [8]:
def compare_resumes(new_resume_text, existing_resumes):
    all_texts = [new_resume_text] + existing_resumes
    vectors = vectorize_text(all_texts)
    cosine_matrix = calculate_cosine_similarity(vectors)
    return cosine_matrix[0, 1:]  # Similarities with all other resumes


In [9]:
def search_online(text_snippet):
    query = '+'.join(text_snippet.split())
    url = f"https://www.google.com/search?q={query}"
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup.find_all('a')


In [10]:
def extract_urls(links):
    urls = []
    for link in links:
        href = link.get('href')
        if href and "url?q=" in href:
            urls.append(href.split("url?q=")[1].split("&")[0])
    return urls


In [11]:
def calculate_plagiarism_score(similarity_scores):
    return sum(similarity_scores) / len(similarity_scores)
def calculate_overall_score(plagiarism_score, num_sources):
    return (plagiarism_score + num_sources) / 2


In [25]:
# Paths to the new resume and existing resumes
new_resume_path = r'C:\Users\thila\Downloads\Untitled Folder\Thilak_SujathaKrishnamurthy_resume (Sdev).pdf'  # Replace with the path to your new resume
existing_resumes_paths = [
    r'C:\Users\thila\Downloads\Untitled Folder\Thilak_SujathaKrishnamurthy_resume (ML).pdf',  # Replace with paths to your existing resumes
    r'C:\Users\thila\Downloads\Untitled Folder\Thilak_SujathaKrishnamurthy_resume (1) (1).docx',
    r'C:\Users\thila\Downloads\Untitled Folder\THILAK SUJATHA KRISHNAMURTHY_SDE.pdf'
    # Add more paths if you have more resumes to compare against
]

def detect_plagiarism_and_score(new_resume_path, existing_resumes_paths):
    print("Starting plagiarism detection...")  # Debugging

    # Extract text from the new resume
    if new_resume_path.endswith('.pdf'):
        new_resume_text = extract_text_from_pdf(new_resume_path)
    elif new_resume_path.endswith('.docx'):
        new_resume_text = extract_text_from_docx(new_resume_path)
    else:
        raise ValueError("Unsupported file format")

    print(f"Extracted new resume text (first 500 characters): {new_resume_text[:500]}")  # Print the first 500 characters for verification

    # Clean the extracted text
    new_resume_text = clean_text(new_resume_text)
    print(f"Cleaned new resume text (first 10 tokens): {new_resume_text[:10]}")  # Print the first 10 tokens for verification

    # Extract and clean text from existing resumes
    existing_resumes_texts = []
    for path in existing_resumes_paths:
        if path.endswith('.pdf'):
            text = extract_text_from_pdf(path)
        elif path.endswith('.docx'):
            text = extract_text_from_docx(path)
        cleaned_text = clean_text(text)
        existing_resumes_texts.append(cleaned_text)
        print(f"Extracted and cleaned existing resume text (first 10 tokens): {cleaned_text[:10]}")  # Print the first 10 tokens for verification

    # Convert lists of tokens back to strings for comparison
    new_resume_text = ' '.join(new_resume_text)
    existing_resumes_texts = [' '.join(text) for text in existing_resumes_texts]

    # Compare the new resume with existing ones
    similarity_scores = compare_resumes(new_resume_text, existing_resumes_texts)
    print(f"Similarity scores: {similarity_scores}")

    plagiarism_score = calculate_plagiarism_score(similarity_scores)
    print(f"Plagiarism score: {plagiarism_score}")

    # Search online for possible sources
    sources_found = []
    for snippet in new_resume_text.split()[:5]:  # Take the first 5 words to search
        links = search_online(snippet)
        urls = extract_urls(links)
        sources_found.extend(urls)
        print(f"Snippet: {snippet}, Number of URLs found: {len(urls)}")

    # Remove duplicates from the sources
    unique_sources = list(set(sources_found))

    # Calculate overall score
    overall_score = calculate_overall_score(plagiarism_score, len(unique_sources))
    print(f"Overall score: {overall_score}")

    return {
        "plagiarism_score": plagiarism_score,
        "num_sources": len(unique_sources),
        "overall_score": overall_score,
        "sources": unique_sources
    }

# Call the main function to get the results
results = detect_plagiarism_and_score(new_resume_path, existing_resumes_paths)

# Output the results
print("Plagiarism Score:", results["plagiarism_score"])
print("Number of Sources Found:", results["num_sources"])
print("Overall Score:", results["overall_score"])
print("Sources Found:")
for source in results["sources"]:
    print(source)


Starting plagiarism detection...
Extracted new resume text (first 500 characters): THILAK SUJATHA KRISHNAMURTHY 
+1 (838) 218-7324 | thilak280400@gmail.com | LinkedIn |Tempe, AZ 

SUMMARY 

Software Engineer with over 2 years of experience specializing in cloud-based and containerized software solutions. 
Successfully led projects improving functionality and scalability using Python, Java, and AWS, Docker, and Kubernetes. 
Proven track record in using JavaScript and optimizing data integration processes. Seeking to leverage skills in software 
development and workflow automati
Cleaned new resume text (first 10 tokens): ['thilak', 'sujatha', 'krishnamurthy', '1', '838', '2187324', 'thilak280400gmailcom', 'linkedin', 'tempe', 'az']
Extracted and cleaned existing resume text (first 10 tokens): ['thilak', 'sujatha', 'krishnamurthy', '1', '838', '2187324', 'thilak280400gmailcom', 'linkedin', 'tempe', 'az']
Extracted and cleaned existing resume text (first 10 tokens): ['thilak', 'sujatha', '

In [ ]:
#Flask App
from flask import Flask, render_template_string, request
import os
from werkzeug.utils import secure_filename

# Initialize Flask app
app = Flask(__name__)

# Set up upload folder
UPLOAD_FOLDER = 'uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Dummy function to simulate plagiarism detection
def detect_plagiarism_and_score(new_resume_path, existing_resumes_paths):
    # Here we would have your existing function implementation.
    # For simplicity, this function will return a dummy result.
    return {
        "plagiarism_score": 0.75,
        "num_sources": 5,
        "overall_score": 0.80,
        "sources": [
            "https://example.com/source1",
            "https://example.com/source2",
            "https://example.com/source3"
        ]
    }

@app.route('/')
def index():
    # Simple HTML form for file upload
    return render_template_string('''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Resume Plagiarism Checker</title>
    </head>
    <body>
        <h1>Upload Your Resume</h1>
        <form action="/upload" method="post" enctype="multipart/form-data">
            <input type="file" name="resume" required>
            <button type="submit">Check Plagiarism</button>
        </form>
    </body>
    </html>
    ''')

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'resume' not in request.files:
        return 'No file uploaded', 400
    
    file = request.files['resume']
    if file.filename == '':
        return 'No file selected', 400
    
    # Save the uploaded file
    filename = secure_filename(file.filename)
    file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
    file.save(file_path)

    # Paths to existing resumes
    existing_resumes_paths = [
        r'C:\Users\thila\Downloads\Untitled Folder\Thilak_SujathaKrishnamurthy_resume (ML).pdf',
        r'C:\Users\thila\Downloads\Untitled Folder\Thilak_SujathaKrishnamurthy_resume (1) (1).docx',
        r'C:\Users\thila\Downloads\Untitled Folder\THILAK SUJATHA KRISHNAMURTHY_SDE.pdf'
    ]

    # Run the plagiarism detection
    results = detect_plagiarism_and_score(file_path, existing_resumes_paths)

    # Display results
    return render_template_string('''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Plagiarism Results</title>
    </head>
    <body>
        <h1>Plagiarism Check Results</h1>
        <p><strong>Plagiarism Score:</strong> {{ results.plagiarism_score }}</p>
        <p><strong>Number of Sources Found:</strong> {{ results.num_sources }}</p>
        <p><strong>Overall Score:</strong> {{ results.overall_score }}</p>
        <h2>Sources Found:</h2>
        <ul>
            {% for source in results.sources %}
                <li><a href="{{ source }}" target="_blank">{{ source }}</a></li>
            {% endfor %}
        </ul>
        <a href="/">Check Another Resume</a>
    </body>
    </html>
    ''', results=results)

# Run the Flask app within the notebook
from werkzeug.serving import run_simple
run_simple('localhost', 5000, app)


 * Running on http://localhost:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Aug/2024 20:12:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Aug/2024 20:12:21] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [13/Aug/2024 20:12:39] "POST /upload HTTP/1.1" 200 -
